In [1]:
import cv2
import numpy as np
from scipy.spatial import distance as dist
from openpyxl import Workbook
from datetime import datetime

# Function to calculate eye aspect ratio (EAR)
def eye_aspect_ratio(eye):
    A = dist.euclidean(eye[1], eye[5])
    B = dist.euclidean(eye[2], eye[4])
    C = dist.euclidean(eye[0], eye[3])
    ear = (A + B) / (2.0 * C)
    return ear

# Load face and eye cascade classifiers
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')

# Initialize video capture
cap = cv2.VideoCapture(0)

# Initialize Excel workbook and worksheet
wb = Workbook()
ws = wb.active
ws.append(["Timestamp", "Attentiveness"])

while True:
    # Read a frame from the video capture
    ret, frame = cap.read()
    if not ret:
        break

    # Convert frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

    for (x,y,w,h) in faces:
        # Draw rectangle around detected face
        cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),2)
        
        # Extract region of interest (ROI) for eyes
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = frame[y:y+h, x:x+w]
        
        # Detect eyes within the face region
        eyes = eye_cascade.detectMultiScale(roi_gray)
        
        # Calculate attentiveness based on eye aspect ratio (EAR)
        attentiveness = 0
        for (ex,ey,ew,eh) in eyes:
            eye = roi_gray[ey:ey+eh, ex:ex+ew]
            ear = eye_aspect_ratio(eye)
            attentiveness += ear
        
        # Normalize attentiveness by the number of eyes
        if len(eyes) > 0:
            attentiveness /= len(eyes)
        
        # Write attentiveness to Excel worksheet
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        ws.append([timestamp, attentiveness])
    
    # Display the frame
    cv2.imshow('frame',frame)
    
    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Save the Excel workbook
wb.save("attentiveness_data.xlsx")
# Release the capture and close OpenCV windows
cap.release()
cv2.destroyAllWindows()